In [7]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize

class Vgg16:
    def __init__(self, images):
        self.parameters = [] #在类的初始化时加入全局列表，将所需共享的参数加载到复用类中
        self.images = images
        self.conv_layers()
        self.fc_layers()
        self.probs = tf.nn.softmax(self.fc8) #输出每个属于各个类别的概率值

    def saver(self):
        return tf.train.Saver()

    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            if i not in [30, 31]: #排除不需要载入的层（只保留最后一个全连阶层fc8）
                sess.run(self.parameters[i].assign(weights[k]))
        print("-----weighes loaded-----")

    def conv(self, name, input_data, out_channel, trainable=False):
        in_channel = input_data.get_shape()[-1]
        with tf.variable_scope(name):
            kernel = tf.get_variable("weights", [3, 3, in_channel, out_channel], dtype=tf.float32, trainable=trainable)
            biases = tf.get_variable("biases", [out_channel], dtype=tf.float32, trainable=trainable)
            conv_res = tf.nn.conv2d(input_data, kernel, [1,1,1,1], padding="SAME")
            res = tf.nn.bias_add(conv_res, biases)
            out = tf.nn.relu(res, name=name)
        self.parameters += [kernel, biases] #将卷积层定义的参数（kernel, biases）加入列表
        return out

    def fullconn(self, name, input_data, out_channel, trainable=True):
        shape = input_data.get_shape().as_list()
        if len(shape) == 4:
            size = shape[-3] * shape[-2] * shape[-1] #获得输入数据各个维度的维数
        else:
            size = shape[1]
        input_data_flat = tf.reshape(input_data, [-1,size])
        with tf.variable_scope(name):
            weights = tf.get_variable(name="weights", shape=[size, out_channel], dtype=tf.float32, trainable=trainable)
            biases = tf.get_variable(name="biases", shape=[out_channel], dtype=tf.float32, trainable=trainable)
            res = tf.matmul(input_data_flat, weights)
            out = tf.nn.relu(tf.nn.bias_add(res, biases))
        self.parameters += [weights, biases] #将全连接层定义的参数（weights, biases）加入列表
        return out

    def conv_layers(self):
        #conv1
        self.conv1_1 = self.conv("conv1re_1", self.images, 64, trainable=False)
        self.conv1_2 = self.conv("conv1_2", self.conv1_1, 64, trainable=False)
        self.pool1 = self.maxpool("poolre1", self.conv1_2)
        #conv2
        self.conv2_1 = self.conv("conv2_1", self.pool1, 128, trainable=False)
        self.conv2_2 = self.conv("convwe2_2", self.conv2_1, 128, trainable=False)
        self.pool2 = self.maxpool("pool2", self.conv2_2)
        #conv3
        self.conv3_1 = self.conv("conv3_1", self.pool2, 256, trainable=False)
        self.conv3_2 = self.conv("convrwe3_2", self.conv3_1, 256, trainable=False)
        self.conv3_3 = self.conv("convrwe3_3", self.conv3_2, 256, trainable=False)
        self.pool3 = self.maxpool("poolre3", self.conv3_3)
        #conv4
        self.conv4_1 = self.conv("conv4_1", self.pool3, 512, trainable=False)
        self.conv4_2 = self.conv("convrwe4_2", self.conv4_1, 512, trainable=False)
        self.conv4_3 = self.conv("convrwe4_3", self.conv4_2, 512, trainable=False)
        self.pool4 = self.maxpool("pool4", self.conv4_3)
        #conv5
        self.conv5_1 = self.conv("conv5_1", self.pool4, 512, trainable=False)
        self.conv5_2 = self.conv("convrwe5_2", self.conv5_1, 512, trainable=False)
        self.conv5_3 = self.conv("conv5_3", self.conv5_2, 512, trainable=False)
        self.pool5 = self.maxpool("poolrwe5", self.conv5_3)

    n_class = 2
    def fc_layers(self):
        global n_class
        self.fc6 = self.fullconn("fc1", self.pool5, 4096, trainable=False)
        self.fc7 = self.fullconn("fc2", self.fc6, 4096, trainable=False)
        self.fc8 = self.fullconn("fc3", self.fc7, self.n_class, trainable=True)

    def maxpool(self, name, input_data):
        out = tf.nn.max_pool(input_data, [1,2,2,1], [1,2,2,1], padding="SAME", name=name)
        return out

tf.reset_default_graph()
    
means = [123.68, 116.799, 103.939] #VGG训练时图像预处理所减均值(RGB三通道)
x = tf.placeholder(tf.float32, [None, 224, 224, 3])

sess = tf.Session()
vgg = Vgg16(x)
fc8_finetuining = vgg.probs

saver = tf.train.Saver()
print("Model restoring...")
saver.restore(sess, "./model/") #恢复最后保存的模型

test_file = r"E:\AI-learn\kaggle\test1\21.jpg" #测试图片集中狗图片中的第21张
#test_file = r"E:\AI-learn\kaggle\test1\92.jpg" #测试图片集中猫图片中的第92张
img = imread(test_file, mode="RGB")
img = imresize(img, (224, 224))
img = img.astype(np.float32) #数组的数据类型转换为float32
#每一通道减去其均值
for c in range(3):
    img[:, :, c] -= means[c]
prob = sess.run(fc8_finetuining, feed_dict={x:[img]})
max_index = np.argmax(prob)

if max_index == 0:
    print("这是一只猫。概率为: %.6f" %prob[:,0])
else:
    print("这是一条狗。概率为: %.6f" %prob[:,1])


Model restoring...
INFO:tensorflow:Restoring parameters from ./model/
这是一条狗。概率为: 0.998811
